# Importing necessary python libraries

In [1]:
from bs4 import BeautifulSoup as soup
import pickle as pkl
import re
import codecs
import os

# Loading HTML file to process

In [2]:
filename=input("Enter the filename in htmls folder: ")
file = open("htmls/"+ filename +".html", "rb")

Enter the filename in htmls folder: html46


# Converting the HTML in soup

In [3]:
data = soup(file)

# Saving all "div" tags into a list

In [4]:
all_divs = []
for div in data.findAll("div"):
    if div.has_attr('class'):
        all_divs.append(div)

# Extracting style from the html

In [5]:
style = data.findAll("style")[0]

In [6]:
st_classes=[]
for line in style:
    stdivs = (line.replace("<!--","").strip().split("\n"))
    st_classes = stdivs

# Creating functions to get css properties from class name 

In [7]:
def getprops(cls, tag):
    di = {}
    for st in st_classes:
        if cls in st and tag in st:
            st1 = st.split("{")[1].replace("}","")
            spl = st1.split(";")
            for sp in spl:
                ssp = sp.split(":")
                if len(ssp) > 1:
                    di[ssp[0]] = ssp[1].replace("\r","")
    return di

# Function to split the hardcoded css properties

In [8]:
def getcprops(stl):
    di = {}
    spl = stl.split(";")
    for s in spl:
        s1 = s.split(":")
        if len(s1) > 1:
            di[s1[0]] = s1[1]
    
    return di

# Function to check if a line has date in it

In [9]:
def checkifdate(txt):
    dt = False
    #clean
    ct=""
    for t in txt:
        if t.isalpha() or t.isdigit() or t==" ":
            ct=ct+t
    
    sp = ct.split(" ")
    for s in sp:
        if len(s) >= 4 and ("18" in s or "19" in s):
            if s.isdigit():
                dt=True
                    
    return dt

# Extracting all "div" tags which have the titles

In [10]:
head_divs=[]
for div in all_divs:
    if div.has_attr("class"):
        di = getprops((div['class'][0]), "div")
        if('font-size' in di):
            if(di['font-size'] == "12.1px") and di['font-style'] == "italic" and div.text[0].isdigit():
                head_divs.append(div)

# Spliting while document into blocks, same as number of titles

In [11]:
blocks = []

for head in head_divs:
    try:
        blocks.append(all_divs[all_divs.index(head):all_divs.index(head_divs[(head_divs.index(head)+1)])])
    except:
        pass

blocks.append(all_divs[all_divs.index(head_divs[-1]):])

# Function to create an empty list of length equal to number of titles

In [12]:
def elist():
    li = []
    for b in blocks:
        li.append("")
    return li

# Breaking each "div" of the block into "span" 

In [13]:
block_ar=[]
for block in blocks:
    div_ar=[]
    for div in block:
        span_ar=[]
        for span in div.findAll("span"):
            span_ar.append(span)
        
        div_ar.append([span_ar,div['style']])
    
    block_ar.append(div_ar)

# Extracting titles

In [14]:
titles=[]
for block in block_ar:
    for div in block:
        if block.index(div) == 0:
            for spans in div[:-1]:
                txt = ""
                for span in spans:
                    txt = txt+span.text
                titles.append(txt)

# Extracting dates

In [15]:
dates=elist()

for block in block_ar:
    for div in block:
        if block.index(div) > 0:
            for spans in div[:-1]:
                txt = ""
                di = getcprops(div[1])
                float(di['left'].replace("px",""))
                if float(di['left'].replace("px","")) >200:
                    for span in spans:
                        txt = txt + span.text

                if checkifdate(txt):
                    dates[block_ar.index(block)] = (txt)

# Extracting cities

In [ ]:
city=elist()
for block in block_ar:
    for div in block:
        if block.index(div) > 0 and block.index(div) < 5:
            for spans in div[:-1]:
                txt = ""
                di = getcprops(div[1])
                lf = float(di['left'].replace("px",""))
                if lf > 200:
                    clss=[]
                    for span in spans:
                        txt = txt + (span.text)
                        dii = getprops(span['class'][0], "span")
                        if span['class'][0] not in clss:
                            clss.append(clss)
                            
                    if len(clss) >= 2 and not checkifdate(txt):
                        city[block_ar.index(block)] = (txt)
                        break

# Extracting text

In [ ]:
wcontent=elist()
for block in block_ar:
    content=[]
    c=0
    rest=False
    last_top=0
    for div in block:
        if block.index(div) > 0:
            di = getcprops(div[1])
            lf = float(di['left'].replace("px",""))
            txt1 = ""
            if lf < 200:
                for spans in div[0]:
                    txt1 = txt1 + spans.text
                    sup = spans.findAll("sup")
                    if len(sup) > 0 and div[0].index(spans) == 0:
                        rest=True
                        last_top = float(di['top'].replace("px",""))
                          
                tp = float(di['top'].replace("px",""))
                if tp-last_top >= 15:
                    rest=False
                        
            if rest==False and not "vol." in txt1.lower() and txt1 not in content and not "[" in txt1 and not "]" in txt1:
                if len(txt1) > 3 and not txt1.isdigit():
                    content.append(txt1)

    wcontent[block_ar.index(block)] = (content)
    content=[]

# Extracting dates at bottom (if any)

In [ ]:
metas=elist()

for content in wcontent:
    m = content[-1]
    
    if checkifdate(m):
        metas[wcontent.index(content)] = m

# Setting type of the document

In [ ]:
types=[]
for title in titles:
    if "letter" in title.lower() or "cable" in title.lower() or "message" in title.lower() or "telegram" in title.lower():
        types.append("private_correspondence")
    else:
        if "speech" in title.lower():
            types.append("speech")
        else:
            if "interview" in title.lower():
                types.append("interview")
            else:
                if "statement" in title.lower():
                    types.append("statement")
                else:
                    if "notes" in title.lower():
                        types.append("column")
                    else:
                        if "discussion" in title.lower():
                            types.append("discussion")
                        else:
                            types.append("")

# Setting format of document

In [ ]:
dformat = elist()
for title in titles:
    i=titles.index(title)
    if "letter" in title.lower() or "cable" in title.lower() or "message" in title.lower() or "diary" in title.lower():
        dformat[i] = "pr"
    else:
        if "interview" in title.lower():
            dformat[i] = "np"

# Merging all components

In [ ]:
final_data=[]
for i in range(len(block_ar)):
    mydict={}
    mydict['date'] = dates[i]
    mydict['content'] = wcontent[i]
    mydict['city'] = city[i]
    mydict['meta'] = metas[i]
    mydict['title'] = titles[i]
    mydict['type'] = types[i]
    mydict['format'] = dformat[i]
    final_data.append(mydict)

In [ ]:
for f in final_data:
    if f['meta'] != "":
        if f['meta'] in f['content']:
            del f['content'][f['content'].index(f['meta'])]
            
    for c in f['content'][5:]:
        if len(c) < 30 and c.strip()[-1] != ".":
            del f['content'][f['content'].index(c)]
        if "From" in c[:6]:
            if c in f['content']:
                del f['content'][f['content'].index(c)]

# Getting date from bottom with regex

In [ ]:
def getbottomdate(btxt):
    dateregex1 = re.compile(r'\d-\d-\d\d\d\d')
    dateregex2 = re.compile(r'\d-\d\d-\d\d\d\d')
    dateregex3 = re.compile(r'\d\d-\d\d-\d\d\d\d')
    dateregex4 = re.compile(r'\d\d-\d-\d\d\d\d')

    dt1 = (dateregex1.search(btxt))
    dt2 = (dateregex2.search(btxt))
    dt3 = (dateregex3.search(btxt))
    dt4 = (dateregex4.search(btxt))

    dt=""

    if dt1 is not None:
        dt = (dt1.group(0))

    if dt2 is not None:
        dt = (dt2.group(0))

    if dt3 is not None:
        dt = (dt3.group(0))

    if dt4 is not None:
        dt = (dt4.group(0))

    if len(dt) >= 8:
        return dt
    else:
        return ""

In [ ]:
for final in final_data:
    if final['meta'] != "":
        fdt = (getbottomdate(final['meta']))
        if fdt != "":
            final['meta'] = fdt
        else:
            final['meta'] = ""

# Function to format date

In [ ]:
def getdate(dttxt):
    mn=""
    dt=""
    yr=""
    
    months=['jan','feb', 'mar', 'apr', 'may', 'jun', 'jul','aug','sept','oct','nov','dec']
    for month in months:
        if month in dttxt.lower():
            mn = str(months.index(month)+1)
            
    yrregex1 = re.compile(r"\d\d\d\d")
    dtregex1 = re.compile(r"\d")
    dtregex2 = re.compile(r"\d\d")
    
    yr1 = yrregex1.search(dttxt)
    if yr1 is not None:
        yr = yr1.group(0)
    
    dt1 = dtregex1.search(dttxt.replace(yr,""))
    dt2 = dtregex2.search(dttxt.replace(yr,""))
    
    if dt1 is not None:
        dt = dt1.group(0)
    if dt2 is not None:
        dt = dt2.group(0)
        
    if (len(dt) == 1 or len(dt) == 2) and (len(mn) == 1 or len(mn) == 2) and (len(yr) == 4):
        return (dt+"-"+mn+"-"+yr)
    else:
        return ""

In [ ]:
for final in final_data:
    rdt = getdate(final['date'])
    if len(rdt) >= 8:
        final['date'] = (rdt)
    else:
        final['date'] = ""

# Replacing empty dates with bottom dates

In [ ]:
for final in final_data:
    if final['date'] == "":
        final['date'] = final['meta']

# Function to get empty dates by average

In [ ]:
def getavdate(dtin):
    if dtin > 0:
        pd = ""
        nd = ""
        pd = (final_data[dtin-1]['date'])
        
        for f in final_data[dtin:]:
            if f['date'] != "":
                nd = (final_data[dtin+1]['date'])
                break
        
        dictpd={}
        dictnd={}
        if pd != "" and nd != "":
            spd = pd.split("-")
            dictpd['dt'] = spd[0]
            dictpd['mn'] = spd[1]
            dictpd['yr'] = spd[2]
            
            snd = nd.split("-")
            dictnd['dt'] = snd[0]
            dictnd['mn'] = snd[1]
            dictnd['yr'] = snd[2]
            
            summ = int(dictpd['dt']) + ((int(dictnd['yr']) - int(dictpd['yr']))*12 + int(dictnd['mn']) - int(dictpd['mn']))*30 + int(dictnd['dt'])
            avg = summ/2

            nnd = ""
            nnm = ""
            nny = ""
            if int(avg) > 30:
                n = int(avg)-30
                if int(dictpd['mn']) < 12:
                    nnm = str(int(dictpd['mn'])+1)
                    nny = str(int(dictpd['yr'])+1)
                else:
                    nnm = str(1)

                nnd = str(int(dictpd['dt'])+n)

            else:
                n=int(avg)
                nnd = str(n)
                nnm = str(dictpd['mn'])
                nny = str(dictpd['yr'])

            return (nnd+"-"+nnm+"-"+nny)
        else:
            return ""
    else:
        nd = ""
        for f in final_data:
            if f['date'] != "":
                nd = (f['date'])
                break
        
        nnd = ""
        nnm = ""
        nny = ""
        
        snd = nd.split("-")
        if int(snd[1]) <= 12: 
            if int(snd[0]) > 5:
                nnd = str(int(snd[0])-5)
                nnm = str(int(snd[1]))
                nny = str(int(snd[2]))
            else:
                nnd = str(24)
                nnm = str(int(snd[1])-1)
                nny = str(int(snd[2]))
        else:
            if int(snd[0]) > 5:
                nnd = str(int(snd[0])-5)
                nnm = str(int(snd[1]))
                nny = str(int(snd[2]))
            else:
                nnd = str(24)
                nnm = str(12)
                nny = str(int(snd[2])-1)
        
        return (nnd+"-"+nnm+"-"+nny)

In [ ]:
for final in final_data:
    if final['date'] == "":
        ad = getavdate(final_data.index(final))
        final['date'] = ad

# Name code generator

In [ ]:
from random import randint
def generatename(txt):
    if len(txt) >= 8:
        sp = txt.split("-")
        dt = sp[0]
        mn = sp[1]
        yr = sp[2]
        rn = randint(100000, 999999)
        if len(dt) == 1:
            dt = "0"+dt
        if len(mn) == 1:
            mn = "0"+mn
            
        return "t"+dt+mn+yr+str(rn)
    else:
        return ""

# Creating text files

In [ ]:
# for fi in final_data:
#     tl=""
#     dt=""
#     ct=""
#     co=""
#     tp=""
#     fo=""
    
#     if fi['title'] != "":
#         tl = fi['title']
#     else:
#         tl = "notitle"
        
#     if fi['date'] != "":
#         dt = fi['date']
#     else:
#         dt="nodate"
        
#     if fi['city'] != "":
#         ct = fi['city']
#     else:
#         ct="nocity"
        
#     if len(fi['content']) > 0:
#         co = "\n".join(fi['content'])
#     else:
#         co = "nocontent"
        
#     if fi['type'] != "":
#         tp = fi['type']
#     else:
#         tp = "enter_type"
        
#     if fi['format'] != "":
#         fo = fi['format']
#     else:
#         fo = "enter_format"
        
#     fx = tl + "\n\n\n" + ct + "\n\n\n" + dt + "\n\n\n" + tp +"\n\n\n" +fo+"\n\n\n" + co
    
#     try:
#         os.makedirs("GandhiTexts/"+filename+"/complete")
#         os.makedirs("GandhiTexts/"+filename+"/incomplete")
#     except:
#         pass
    
#     fn=tl
#     if fi['date'] != "":
#         fn = generatename(fi['date'])
#     if tl != "nptitle" and ct != "nocity" and dt != "nodate" and tp != "enter_type" and tp != "speech" and tp != "statement" and fo != "enter_format":
#         file = codecs.open("GandhiTexts/"+filename+"/complete/"+fn+".txt", "w", "utf-8")
#         file.write(fx)
#         file.close()
#     else:
#         file = codecs.open("GandhiTexts/"+filename+"/incomplete/"+fn+".txt", "w", "utf-8")
#         file.write(fx)
#         file.close()

# Creating final Excel Sheet

In [ ]:
import xlwt 
from xlwt import Workbook

In [ ]:
wb = Workbook()  

try:
    os.makedirs("GandhiTexts")
except:
    pass

for fi in final_data:
    fn=fi['title']
    if fi['date'] != "":
        fn = generatename(fi['date'])
    
    mind = final_data.index(fi)
    
    sheet1 = wb.add_sheet(str(mind))
    
    co = "\n".join(fi['content'][:20])
    
    sheet1.write(0, 0, fn)
    sheet1.write(0, 1, fi['title'])
    sheet1.write(0, 2, fi['city'])
    sheet1.write(0, 3, fi['date'])
    sheet1.write(0, 4, fi['type'])
    sheet1.write(0, 5, fi['format'])
    
    for line in fi['content']:
        sheet1.write(fi['content'].index(line), 6, line)

wb.save('GandhiTexts/'+ filename +'.xls')